In [1]:
import pandas as pd
import numpy as np

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

pd.set_option('display.max_columns', None)

In [2]:
manga = pd.read_csv('manga.csv')
manga

,id,title,media_type,mean,num_scoring_users,status,num_volumes,num_chapters,start_date,end_date,num_list_users,popularity,num_favorites,rank,genres,authors,synopsis,nsfw,created_at,updated_at,main_picture_medium,main_picture_large,alternative_titles_en,alternative_titles_ja,alternative_titles_synonyms
0,23390,Shingeki no Kyojin,manga,8.57,361834,finished,34.0,141.0,2009-09-09,2021-04-09,588155,1,66856,87.0,"['Action', 'Award Winning', 'Drama', 'Gore', '...","['Isayama, Hajime (Story & Art)']","Hundreds of years ago, horrifying creatures wh...",white,1970-01-01 00:00:00+00:00,2022-04-18 05:10:58+00:00,https://api-cdn.myanimelist.net/images/manga/2...,https://api-cdn.myanimelist.net/images/manga/2...,Attack on Titan,進撃の巨人,[]
1,2,Berserk,manga,9.45,266303,currently_publishing,NaN,NaN,1989-08-25,NaN,546977,2,103023,1.0,"['Action', 'Adventure', 'Award Winning', 'Dram...","['Miura, Kentarou (Story & Art)', 'Studio Gaga...","Guts, a former mercenary now known as the ""Bla...",white,1970-01-01 00:00:00+00:00,2022-06-23 08:30:44+00:00,https://api-cdn.myanimelist.net/images/manga/1...,https://api-cdn.myanimelist.net/images/manga/1...,Berserk,ベルセルク,['Berserk: The Prototype']
2,13,One Piece,manga,9.20,303735,currently_publishing,NaN,NaN,1997-07-22,NaN,497765,3,98931,3.0,"['Action', 'Adventure', 'Fantasy', 'Shounen']","['Oda, Eiichiro (Story & Art)']","Gol D. Roger, a man referred to as the ""Pirate...",white,1970-01-01 00:00:00+00:00,2022-04-18 04:48:35+00:00,https://api-cdn.myanimelist.net/images/manga/2...,https://api-cdn.myanimelist.net/images/manga/2...,One Piece,ONE PIECE,[]
3,33327,Tokyo Ghoul,manga,8.52,235494,finished,14.0,144.0,2011-09-08,2014-09-18,433915,4,40166,114.0,"['Action', 'Fantasy', 'Gore', 'Horror', 'Psych...","['Ishida, Sui (Story & Art)']",Lurking within the shadows of Tokyo are fright...,white,1970-01-01 00:00:00+00:00,2022-04-18 03:30:48+00:00,https://api-cdn.myanimelist.net/images/manga/3...,https://api-cdn.myanimelist.net/images/manga/3...,Tokyo Ghoul,東京喰種トーキョーグール,['Toukyou Kushu']
4,116778,Chainsaw Man,manga,8.73,262318,currently_publishing,NaN,NaN,2018-12-03,NaN,422419,5,58016,44.0,"['Action', 'Adventure', 'Award Winning', 'Gore...","['Fujimoto, Tatsuki (Story & Art)']",Denji has a simple dream—to live a happy and p...,white,1970-01-01 00:00:00+00:00,2022-07-12 15:40:41+00:00,https://api-cdn.myanimelist.net/images/manga/3...,https://api-cdn.myanimelist.net/images/manga/3...,Chainsaw Man,チェンソーマン,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59762,149239,How a Mage Welcomes Death,manhwa,NaN,0,currently_publishing,NaN,NaN,2022-02-11,NaN,6,59763,0,20665.0,"['Fantasy', 'Reincarnation']","['Park, Jeff (Art)', 'Ju, Eunseol (Story)']","For the past 300 years, Ente has fought agains...",white,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,https://api-cdn.myanimelist.net/images/manga/1...,https://api-cdn.myanimelist.net/images/manga/1...,How a Mage Welcomes Death,마법사가 죽음을 맞이하는 방법,['The Way the Mage Faces Death']
59763,146492,Kubikoke Kisakitan: Koneko no Shitone,manga,NaN,0,finished,1.0,7.0,2020-11-27,2021-11-26,3,59765,0,NaN,"['Boys Love', 'Erotica', 'Fantasy']","['Suzuki, Ami (Story)', 'Rubiru (Art']",NaN,gray,1970-01-01 00:00:00+00:00,2022-07-08 19:16:24+00:00,https://api-cdn.myanimelist.net/images/manga/1...,https://api-cdn.myanimelist.net/images/manga/1...,NaN,九尾狐家妃譚 ~仔猫の褥~,[]
59764,148799,"Yuri no Aida ni Hasamareta Watashi ga, Ikioi d...",light_novel,NaN,0,currently_publishing,NaN,NaN,2021-11-25,NaN,3,59766,0,20510.0,"['Girls Love', 'Love Polygon', 'School']","['Shiina, Kuro (Art)', 'Toshizou (Story']",Yuna Momose and Rinka Aiba were made for each ...,gray,1970-01-01 00:00:00+00:00,2022-07-11 18:39:26+00:00,https://api-cdn.myanimelist.net/images/manga/1...,https://api-cdn.myanimelist.net/images/manga/1...,Yuri Tama: From Third Wheel to Trifecta,百合の間に挟まれたわたしが、勢いで二股してしまった話,['Yuri no Aida ni Hasamatteshimatta Onna']
59765,122349,Nth Romance,manhwa,NaN,

In [3]:
manga=manga.dropna(subset=['title', 'mean', 'main_picture_medium'])

# Menghitung Score

In [4]:
C = manga['mean'].mean()

In [5]:
m = manga['num_scoring_users'].quantile(0.75)

In [6]:
manga = manga.loc[manga['num_scoring_users'] >= m]
manga.shape

(5457, 25)

In [7]:
def weighted_rating(x, m=m, C=C):
    v = x['num_scoring_users']
    R = x['mean']
    # Calculation based on the IMDB formula
    return np.round(((v/(v+m) * R) + (m/(m+v) * C)), 2)

In [8]:
manga['score'] = manga.apply(weighted_rating, axis=1)

In [9]:
manga.drop(['num_scoring_users', 'num_list_users', 'num_favorites', 'alternative_titles_synonyms'], axis=1, inplace=True)

In [10]:
manga.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5457 entries, 0 to 8703
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     5457 non-null   int64  
 1   title                  5457 non-null   object 
 2   media_type             5457 non-null   object 
 3   mean                   5457 non-null   float64
 4   status                 5457 non-null   object 
 5   num_volumes            4267 non-null   float64
 6   num_chapters           4509 non-null   float64
 7   start_date             5411 non-null   object 
 8   end_date               4449 non-null   object 
 9   popularity             5457 non-null   int64  
 10  rank                   4651 non-null   float64
 11  genres                 5457 non-null   object 
 12  authors                5457 non-null   object 
 13  synopsis               5402 non-null   object 
 14  nsfw                   5457 non-null   object 
 15  crea

In [11]:
manga.isna().sum()

id                          0
title                       0
media_type                  0
mean                        0
status                      0
num_volumes              1190
num_chapters              948
start_date                 46
end_date                 1008
popularity                  0
rank                      806
genres                      0
authors                     0
synopsis                   55
nsfw                        0
created_at                  0
updated_at                  0
main_picture_medium         0
main_picture_large          0
alternative_titles_en    2840
alternative_titles_ja      36
score                       0
dtype: int64

---

In [12]:
# cek data duplikat
manga.duplicated().any()

False

---

In [13]:
list_col = ['genres', 'authors']
mapping = [("[", ""), ("]", ""), ("'", ""), ('"', '')]
for i, j in mapping:
    for k in list_col:
        manga[k] = manga[k].apply(lambda x: x.replace(i, j))

In [14]:
manga.authors.sample(5)

6723    Kajiyama, Mika (Art), Hashimoto, Kanari (Story)
4867                       Okuda, Hitoshi (Story & Art)
3357                     Wazarai, Shizuya (Story & Art)
969                            Yonezawa, Honobu (Story)
58                         Hara, Yasuhisa (Story & Art)
Name: authors, dtype: object

In [15]:
soup = manga[['title', 'genres', 'authors', 'synopsis', 'nsfw' , 'alternative_titles_en']].reset_index(drop=True)
soup

,title,genres,authors,synopsis,nsfw,alternative_titles_en
0,Shingeki no Kyojin,"Action, Award Winning, Drama, Gore, Military, ...","Isayama, Hajime (Story & Art)","Hundreds of years ago, horrifying creatures wh...",white,Attack on Titan
1,Berserk,"Action, Adventure, Award Winning, Drama, Fanta...","Miura, Kentarou (Story & Art), Studio Gaga (Art","Guts, a former mercenary now known as the ""Bla...",white,Berserk
2,One Piece,"Action, Adventure, Fantasy, Shounen","Oda, Eiichiro (Story & Art)","Gol D. Roger, a man referred to as the ""Pirate...",white,One Piece
3,Tokyo Ghoul,"Action, Fantasy, Gore, Horror, Psychological, ...","Ishida, Sui (Story & Art)",Lurking within the shadows of Tokyo are fright...,white,Tokyo Ghoul
4,Chainsaw Man,"Action, Adventure, Award Winning, Gore, Shounen","Fujimoto, Tatsuki (Story & Art)",Denji has a simple dream—to live a happy and p...,white,Chainsaw Man
...,...,...,...,...,...,...
5452,Boys Be Zombitious,"Action, Gore, Horror, Seinen","Kodama, Yuuki (Story & Art)","During a zombie apocalypse, he looks for a per...",white,NaN
5453,Gaki no Koro kara,"Drama, Seinen","Isshiki, Makoto (Story & Art)",1. Shou-chan\n2. Yarou nanka nya Wakarumai!\n3...,white,NaN
5454,Yoake Mae,"Ecchi, Josei, Romance","Yamada, Kana (Story & Art)",A woman and her sexual adventures with her bro...,gray,NaN
5455,Zannen nagara Chigaimasu.,"Romance, Shoujo","Aoki, Kotomi (Story & Art)","Following what seems like a mundane encounter,...",white,NaN


In [16]:
unwanted_str = [('Story & Art', ''), ('Story', ''), ('Art', ''), ('(', ''), (')', '')]
for i, j in unwanted_str:
    soup['authors'] = soup['authors'].str.replace(i, j)

In [17]:
mapping_soup = [(" ", ""), (",", " ")]
for i, j in mapping_soup:
    for k in ['genres', 'authors']:
        soup[k] = soup[k].apply(lambda x: x.replace(i, j))

In [18]:
for i in ['genres', 'authors']:
    manga.loc[manga[i]=="", i] = np.nan
    soup.loc[soup[i]=="", i] = np.nan

In [19]:
manga.isna().sum()

id                          0
title                       0
media_type                  0
mean                        0
status                      0
num_volumes              1190
num_chapters              948
start_date                 46
end_date                 1008
popularity                  0
rank                      806
genres                      1
authors                     1
synopsis                   55
nsfw                        0
created_at                  0
updated_at                  0
main_picture_medium         0
main_picture_large          0
alternative_titles_en    2840
alternative_titles_ja      36
score                       0
dtype: int64

In [20]:
soup.isna().sum()

title                       0
genres                      1
authors                     1
synopsis                   55
nsfw                        0
alternative_titles_en    2840
dtype: int64

---

In [21]:
genres_null_index = list(soup.loc[soup.genres.isna() &
                                  ~soup.authors.isna() &
                                  ~soup.alternative_titles_en.isna()
                                 ].index)

authors_null_index = list(soup.loc[~soup.genres.isna() &
                                   soup.authors.isna() &
                                   ~soup.alternative_titles_en.isna()
                                  ].index)
entitle_null_index = list(soup.loc[~soup.genres.isna() &
                                   ~soup.authors.isna() &
                                   soup.alternative_titles_en.isna()
                                  ].index)

genres_authors_null_index = list(soup.loc[soup.genres.isna() &
                                          soup.authors.isna() &
                                          ~soup.alternative_titles_en.isna()].index)
genres_entitle_null_index = list(soup.loc[soup.genres.isna() &
                                          ~soup.authors.isna() &
                                          soup.alternative_titles_en.isna()].index)
authors_entitle_null_index = list(soup.loc[~soup.genres.isna() &
                                           soup.authors.isna() &
                                           soup.alternative_titles_en.isna()].index)

null_3_index = list(soup.loc[soup.genres.isna() &
                             soup.authors.isna() &
                             soup.alternative_titles_en.isna()].index)

In [22]:
union_null_index = list(set(genres_null_index) | set(authors_null_index) |
                        set(entitle_null_index) | set(genres_authors_null_index) |
                        set(genres_entitle_null_index) | set(authors_entitle_null_index) |
                        set(null_3_index)
                       )
non_null_index = list(set(soup.index) - set(union_null_index))

In [23]:
union = list(set(genres_null_index) & set(authors_null_index) &
                        set(entitle_null_index) & set(genres_authors_null_index) &
                        set(genres_entitle_null_index) & set(authors_entitle_null_index) &
                        set(null_3_index)
                       )

In [24]:
soup['soup'] = np.nan

In [25]:
# 3 coloumn tidak null
soup.soup.iloc[non_null_index] = soup.title.iloc[non_null_index] + ' ' + \
soup.genres.iloc[non_null_index] + ' ' + soup.authors.iloc[non_null_index] + ' ' +\
soup.nsfw.iloc[non_null_index] + ' ' + soup.alternative_titles_en.iloc[non_null_index]
# hanya genres yang null
soup.soup.iloc[genres_null_index] = soup.title.iloc[genres_null_index] + ' ' +\
soup.authors.iloc[genres_null_index] + ' ' + \
soup.nsfw.iloc[genres_null_index] + ' ' + soup.alternative_titles_en.iloc[genres_null_index]
# hanya authors yang null
soup.soup.iloc[authors_null_index] = soup.title.iloc[authors_null_index] + ' ' + \
soup.genres.iloc[authors_null_index] + ' ' + \
soup.nsfw.iloc[authors_null_index] + ' ' + soup.alternative_titles_en.iloc[authors_null_index]
# hanya alter title en yang null
soup.soup.iloc[entitle_null_index] = soup.title.iloc[entitle_null_index] + ' ' + \
soup.genres.iloc[entitle_null_index] + ' ' + soup.authors.iloc[entitle_null_index] + ' ' +\
soup.nsfw.iloc[entitle_null_index]
# genres dan authors null
soup.soup.iloc[genres_authors_null_index] = soup.title.iloc[genres_authors_null_index] + ' ' + \
soup.nsfw.iloc[genres_authors_null_index] + ' ' + soup.alternative_titles_en.iloc[genres_authors_null_index]
# genres dan alter title en null
soup.soup.iloc[genres_entitle_null_index] = soup.title.iloc[genres_entitle_null_index] + ' ' + \
soup.authors.iloc[genres_entitle_null_index] + ' ' + soup.nsfw.iloc[genres_entitle_null_index]
# authors dan alter title en null
soup.soup.iloc[authors_entitle_null_index] = soup.title.iloc[authors_entitle_null_index] + ' ' + \
soup.genres.iloc[authors_entitle_null_index] + ' ' + soup.nsfw.iloc[authors_entitle_null_index]
# 3 kolom null
soup.soup.iloc[null_3_index] = soup.title.iloc[null_3_index] + ' ' + soup.nsfw.iloc[null_3_index]

In [26]:
soup=soup['soup']
soup = soup.apply(lambda x: x.lower())
soup

0       shingeki no kyojin action awardwinning drama g...
1       berserk action adventure awardwinning drama fa...
2       one piece action adventure fantasy shounen oda...
3       tokyo ghoul action fantasy gore horror psychol...
4       chainsaw man action adventure awardwinning gor...
                              ...                        
5452    boys be zombitious action gore horror seinen k...
5453    gaki no koro kara drama seinen isshiki makoto ...
5454       yoake mae ecchi josei romance yamada kana gray
5455    zannen nagara chigaimasu. romance shoujo aoki ...
5456    f-ken combatsports comedy school shounen sport...
Name: soup, Length: 5457, dtype: object

In [27]:
soup.to_csv('manga_soup_genres_authors_based.csv', index=False)

In [28]:
soup = pd.read_csv('manga_soup_genres_authors_based.csv')

In [29]:
soup

,soup
0,shingeki no kyojin action awardwinning drama g...
1,berserk action adventure awardwinning drama fa...
2,one piece action adventure fantasy shounen oda...
3,tokyo ghoul action fantasy gore horror psychol...
4,chainsaw man action adventure awardwinning gor...
...,...
5452,boys be zombitious action gore horror seinen k...
5453,gaki no koro kara drama seinen isshiki makoto ...
5454,yoake mae ecchi josei romance yamada kana gray
5455,zannen nagara chigaimasu. romance shoujo aoki ...


---

## Manga Display

In [30]:
manga.head()

,id,title,media_type,mean,status,num_volumes,num_chapters,start_date,end_date,popularity,rank,genres,authors,synopsis,nsfw,created_at,updated_at,main_picture_medium,main_picture_large,alternative_titles_en,alternative_titles_ja,score
0,23390,Shingeki no Kyojin,manga,8.57,finished,34.0,141.0,2009-09-09,2021-04-09,1,87.0,"Action, Award Winning, Drama, Gore, Military, ...","Isayama, Hajime (Story & Art)","Hundreds of years ago, horrifying creatures wh...",white,1970-01-01 00:00:00+00:00,2022-04-18 05:10:58+00:00,https://api-cdn.myanimelist.net/images/manga/2...,https://api-cdn.myanimelist.net/images/manga/2...,Attack on Titan,進撃の巨人,8.57
1,2,Berserk,manga,9.45,currently_publishing,NaN,NaN,1989-08-25,NaN,2,1.0,"Action, Adventure, Award Winning, Drama, Fanta...","Miura, Kentarou (Story & Art), Studio Gaga (Art","Guts, a former mercenary now known as the ""Bla...",white,1970-01-01 00:00:00+00:00,2022-06-23 08:30:44+00:00,https://api-cdn.myanimelist.net/images/manga/1...,https://api-cdn.myanimelist.net/images/manga/1...,Berserk,ベルセルク,9.44
2,13,One Piece,manga,9.20,currently_publishing,NaN,NaN,1997-07-22,NaN,3,3.0,"Action, Adventure, Fantasy, Shounen","Oda, Eiichiro (Story & Art)","Gol D. Roger, a man referred to as the ""Pirate...",white,1970-01-01 00:00:00+00:00,2022-04-18 04:48:35+00:00,https://api-cdn.myanimelist.net/images/manga/2...,https://api-cdn.myanimelist.net/images/manga/2...,One Piece,ONE PIECE,9.19
3,33327,Tokyo Ghoul,manga,8.52,finished,14.0,144.0,2011-09-08,2014-09-18,4,114.0,"Action, Fantasy, Gore, Horror, Psychological, ...","Ishida, Sui (Story & Art)",Lurking within the shadows of Tokyo are fright...,white,1970-01-01 00:00:00+00:00,2022-04-18 03:30:48+00:00,https://api-cdn.myanimelist.net/images/manga/3...,https://api-cdn.myanimelist.net/images/manga/3...,Tokyo Ghoul,東京喰種トーキョーグール,8.51
4,116778,Chainsaw Man,manga,8.73,currently_publishing,NaN,NaN,2018-12-03,NaN,5,44.0,"Action, Adventure, Award Winning, Gore, Shounen","Fujimoto, Tatsuki (Story & Art)",Denji has a simple dream—to live a happy and p...,white,1970-01-01 00:00:00+00:00,2022-07-12 15:40:41+00:00,https://api-cdn.myanimelist.net/images/manga/3...,https://api-cdn.myanimelist.net/images/manga/3...,Chainsaw Man,チェンソーマン,8.72


In [31]:
manga.drop(['id', 'mean', 'main_picture_large'], axis=1, inplace=True)

In [32]:
manga.isna().sum()

title                       0
media_type                  0
status                      0
num_volumes              1190
num_chapters              948
start_date                 46
end_date                 1008
popularity                  0
rank                      806
genres                      1
authors                     1
synopsis                   55
nsfw                        0
created_at                  0
updated_at                  0
main_picture_medium         0
alternative_titles_en    2840
alternative_titles_ja      36
score                       0
dtype: int64

In [33]:
list_unknown = ['num_volumes', 'num_chapters', 'start_date', 'genres', 'authors']
manga[list_unknown] = manga[list_unknown].fillna('Unknown')
manga['end_date'] = manga['end_date'].fillna('?')

In [34]:
list_ = ['rank', 'alternative_titles_en', 'alternative_titles_ja']
manga[list_] = manga[list_].fillna('-')
manga['synopsis'] = manga['synopsis'].fillna('No synopsis for this title')

In [35]:
status_str = [('finished', 'Finished'), ('currently_publishing', 'Publishing'), 
                ('on_hiatus', 'On Hiatus'), ('discontinued', 'Discontinued')]
for i, j in status_str:
    manga['status'] = manga['status'].str.replace(i, j)

In [36]:
type_str = [('manga', 'Manga'), ('one_shot', 'One Shot'), ('manhwa', 'Manhwa'), 
            ('light_novel', 'Light Novel'), ('manhua', 'Manhua'), ('novel', 'Novel')]
for i, j in type_str:
    manga['media_type'] = manga['media_type'].str.replace(i, j)

In [37]:
manga.isna().sum()

title                    0
media_type               0
status                   0
num_volumes              0
num_chapters             0
start_date               0
end_date                 0
popularity               0
rank                     0
genres                   0
authors                  0
synopsis                 0
nsfw                     0
created_at               0
updated_at               0
main_picture_medium      0
alternative_titles_en    0
alternative_titles_ja    0
score                    0
dtype: int64

In [38]:
manga

,title,media_type,status,num_volumes,num_chapters,start_date,end_date,popularity,rank,genres,authors,synopsis,nsfw,created_at,updated_at,main_picture_medium,alternative_titles_en,alternative_titles_ja,score
0,Shingeki no Kyojin,Manga,Finished,34.0,141.0,2009-09-09,2021-04-09,1,87.0,"Action, Award Winning, Drama, Gore, Military, ...","Isayama, Hajime (Story & Art)","Hundreds of years ago, horrifying creatures wh...",white,1970-01-01 00:00:00+00:00,2022-04-18 05:10:58+00:00,https://api-cdn.myanimelist.net/images/manga/2...,Attack on Titan,進撃の巨人,8.57
1,Berserk,Manga,Publishing,Unknown,Unknown,1989-08-25,?,2,1.0,"Action, Adventure, Award Winning, Drama, Fanta...","Miura, Kentarou (Story & Art), Studio Gaga (Art","Guts, a former mercenary now known as the ""Bla...",white,1970-01-01 00:00:00+00:00,2022-06-23 08:30:44+00:00,https://api-cdn.myanimelist.net/images/manga/1...,Berserk,ベルセルク,9.44
2,One Piece,Manga,Publishing,Unknown,Unknown,1997-07-22,?,3,3.0,"Action, Adventure, Fantasy, Shounen","Oda, Eiichiro (Story & Art)","Gol D. Roger, a man referred to as the ""Pirate...",white,1970-01-01 00:00:00+00:00,2022-04-18 04:48:35+00:00,https://api-cdn.myanimelist.net/images/manga/2...,One Piece,ONE PIECE,9.19
3,Tokyo Ghoul,Manga,Finished,14.0,144.0,2011-09-08,2014-09-18,4,114.0,"Action, Fantasy, Gore, Horror, Psychological, ...","Ishida, Sui (Story & Art)",Lurking within the shadows of Tokyo are fright...,white,1970-01-01 00:00:00+00:00,2022-04-18 03:30:48+00:00,https://api-cdn.myanimelist.net/images/manga/3...,Tokyo Ghoul,東京喰種トーキョーグール,8.51
4,Chainsaw Man,Manga,Publishing,Unknown,Unknown,2018-12-03,?,5,44.0,"Action, Adventure, Award Winning, Gore, Shounen","Fujimoto, Tatsuki (Story & Art)",Denji has a simple dream—to live a happy and p...,white,1970-01-01 00:00:00+00:00,2022-07-12 15:40:41+00:00,https://api-cdn.myanimelist.net/images/manga/3...,Chainsaw Man,チェンソーマン,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,Boys Be Zombitious,One Shot,Finished,Unknown,1.0,2016-02-29,2016-02-29,7902,12516.0,"Action, Gore, Horror, Seinen","Kodama, Yuuki (Story & Art)","During a zombie apocalypse, he looks for a per...",white,1970-01-01 00:00:00+00:00,2022-03-30 07:24:11+00:00,https://api-cdn.myanimelist.net/images/manga/3...,-,BOYS BE ZOMBITIOUS,6.78
8040,Gaki no Koro kara,Manga,Finished,1.0,13.0,1990-07-01,2004-08-01,8043,4073.0,"Drama, Seinen","Isshiki, Makoto (Story & Art)",1. Shou-chan\n2. Yarou nanka nya Wakarumai!\n3...,white,1970-01-01 00:00:00+00:00,2022-02-10 06:34:59+00:00,https://api-cdn.myanimelist.net/images/manga/2...,-,―ガキの頃から― 一色まこと短編集,7.14
8073,Yoake Mae,One Shot,Finished,Unknown,1.0,2005-01-01,2005-01-01,8074,15858.0,"Ecchi, Josei, Romance","Yamada, Kana (Story & Art)",A woman and her sexual adventures with her bro...,gray,1970-01-01 00:00:00+00:00,2021-03-15 02:29:28+00:00,https://api-cdn.myanimelist.net/images/manga/2...,-,ヨアケマエ,6.03
8476,Zannen nagara Chigaimasu.,One Shot,Finished,Unknown,1.0,2014-12-10,2014-12-10,8477,15882.0,"Romance, Shoujo","Aoki, Kotomi (Story & Art)","Following what seems like a mundane encounter,...",white,1970-01-01 00:00:00+00:00,2022-04-16 02:28:58+00:00,https://api-cdn.myanimelist.net/images/manga/1...,-,残念ながら違います。,4.94


In [39]:
manga.to_csv('manga_display.csv', index=False)

In [40]:
manga = pd.read_csv('manga_display.csv')

----

In [41]:
soup_read = pd.read_csv('manga_soup_genres_authors_based.csv')
soup_read

,soup
0,shingeki no kyojin action awardwinning drama g...
1,berserk action adventure awardwinning drama fa...
2,one piece action adventure fantasy shounen oda...
3,tokyo ghoul action fantasy gore horror psychol...
4,chainsaw man action adventure awardwinning gor...
...,...
5452,boys be zombitious action gore horror seinen k...
5453,gaki no koro kara drama seinen isshiki makoto ...
5454,yoake mae ecchi josei romance yamada kana gray
5455,zannen nagara chigaimasu. romance shoujo aoki ...
